In [30]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from numpy import random
from sklearn import svm
from __future__ import print_function
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from random import choice
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import validation_curve
from pylab import *  
from matplotlib.ticker import MultipleLocator, FormatStrFormatter  
from collections import Counter
from sklearn.calibration import CalibratedClassifierCV
%matplotlib inline

In [44]:
# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (16.0, 9.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
train_data=np.load("D:/GitHub/Neural-Network-Theory-and-Applications-Homework-SJTU2018/Homework Assignment 2/data_hw2/train_data.npy")
train_label=np.load("D:/GitHub/Neural-Network-Theory-and-Applications-Homework-SJTU2018/Homework Assignment 2/data_hw2/train_label.npy")
test_data=np.load("D:/GitHub/Neural-Network-Theory-and-Applications-Homework-SJTU2018/Homework Assignment 2/data_hw2/test_data.npy")
test_label=np.load("D:/GitHub/Neural-Network-Theory-and-Applications-Homework-SJTU2018/Homework Assignment 2/data_hw2/test_label.npy")

##原来的标签是{-1,0,1}，现在变为三堆样本标签，正样本是0，负样本是1

In [4]:
train_label_0=np.where(train_label == 0, 0, 1)
train_label_1=np.where(train_label == 1, 0, 1)
train_label_negative1=np.where(train_label == -1, 0, 1)

##建立正负样本索引

In [41]:
train_index_0_0=[]
train_index_0_1=[]
train_index_1_0=[]
train_index_1_1=[]
train_index_negative1_0=[]
train_index_negative1_1=[]
for i in range(37367):
    if train_label_0[i] == 0:
        train_index_0_0.append(i)
    else:
        train_index_0_1.append(i)
for i in range(37367):
    if train_label_1[i] == 0:
        train_index_1_0.append(i)
    else:
        train_index_1_1.append(i)
for i in range(37367):
    if train_label_negative1[i] == 0:
        train_index_negative1_0.append(i)
    else:
        train_index_negative1_1.append(i)

##将索引随机打乱

In [42]:
np.random.shuffle(train_index_0_0)
np.random.shuffle(train_index_0_1)
np.random.shuffle(train_index_1_0)
np.random.shuffle(train_index_1_1)
np.random.shuffle(train_index_negative1_0)
np.random.shuffle(train_index_negative1_1)

#### 实现了论文中的正负索引随机分堆，共3个二分类问题，每个二分类问题正样本2堆，负样本4堆

In [68]:
train_index_0_0_1st=train_index_0_0[:int(0.5*len(train_index_0_0))]
train_index_0_0_2st=train_index_0_0[int(0.5*len(train_index_0_0)):]
train_index_0_1_1st=train_index_0_1[:int(0.25*len(train_index_0_1))]
train_index_0_1_2st=train_index_0_1[int(0.25*len(train_index_0_1)):int(0.5*len(train_index_0_1))]
train_index_0_1_3st=train_index_0_1[int(0.5*len(train_index_0_1)):int(0.75*len(train_index_0_1))]
train_index_0_1_4st=train_index_0_1[int(0.75*len(train_index_0_1)):]

train_index_1_0_1st=train_index_1_0[:int(0.5*len(train_index_1_0))]
train_index_1_0_2st=train_index_1_0[int(0.5*len(train_index_1_0)):]
train_index_1_1_1st=train_index_1_1[:int(0.25*len(train_index_1_1))]
train_index_1_1_2st=train_index_1_1[int(0.25*len(train_index_1_1)):int(0.5*len(train_index_1_1))]
train_index_1_1_3st=train_index_1_1[int(0.5*len(train_index_1_1)):int(0.75*len(train_index_1_1))]
train_index_1_1_4st=train_index_1_1[int(0.75*len(train_index_1_1)):]

train_index_negative1_0_1st=train_index_negative1_0[:int(0.5*len(train_index_negative1_0))]
train_index_negative1_0_2st=train_index_negative1_0[int(0.5*len(train_index_negative1_0)):]
train_index_negative1_1_1st=train_index_negative1_1[:int(0.25*len(train_index_negative1_1))]
train_index_negative1_1_2st=train_index_negative1_1[int(0.25*len(train_index_negative1_1)):int(0.5*len(train_index_negative1_1))]
train_index_negative1_1_3st=train_index_negative1_1[int(0.5*len(train_index_negative1_1)):int(0.75*len(train_index_negative1_1))]
train_index_negative1_1_4st=train_index_negative1_1[int(0.75*len(train_index_negative1_1)):]

##实现了论文中的正负索引组合，生成8堆混合样本

In [69]:
train_index_0_11=np.concatenate((train_index_0_0_1st,train_index_0_1_1st)) 
train_index_0_12=np.concatenate((train_index_0_0_1st,train_index_0_1_2st)) 
train_index_0_13=np.concatenate((train_index_0_0_1st,train_index_0_1_3st)) 
train_index_0_14=np.concatenate((train_index_0_0_1st,train_index_0_1_4st)) 
train_index_0_21=np.concatenate((train_index_0_0_2st,train_index_0_1_1st)) 
train_index_0_22=np.concatenate((train_index_0_0_2st,train_index_0_1_2st)) 
train_index_0_23=np.concatenate((train_index_0_0_2st,train_index_0_1_3st)) 
train_index_0_24=np.concatenate((train_index_0_0_2st,train_index_0_1_4st)) 

train_index_1_11=np.concatenate((train_index_1_0_1st,train_index_1_1_1st)) 
train_index_1_12=np.concatenate((train_index_1_0_1st,train_index_1_1_2st)) 
train_index_1_13=np.concatenate((train_index_1_0_1st,train_index_1_1_3st)) 
train_index_1_14=np.concatenate((train_index_1_0_1st,train_index_1_1_4st)) 
train_index_1_21=np.concatenate((train_index_1_0_2st,train_index_1_1_1st)) 
train_index_1_22=np.concatenate((train_index_1_0_2st,train_index_1_1_2st)) 
train_index_1_23=np.concatenate((train_index_1_0_2st,train_index_1_1_3st)) 
train_index_1_24=np.concatenate((train_index_1_0_2st,train_index_1_1_4st)) 

train_index_negative1_11=np.concatenate((train_index_negative1_0_1st,train_index_negative1_1_1st)) 
train_index_negative1_12=np.concatenate((train_index_negative1_0_1st,train_index_negative1_1_2st)) 
train_index_negative1_13=np.concatenate((train_index_negative1_0_1st,train_index_negative1_1_3st)) 
train_index_negative1_14=np.concatenate((train_index_negative1_0_1st,train_index_negative1_1_4st)) 
train_index_negative1_21=np.concatenate((train_index_negative1_0_2st,train_index_negative1_1_1st)) 
train_index_negative1_22=np.concatenate((train_index_negative1_0_2st,train_index_negative1_1_2st)) 
train_index_negative1_23=np.concatenate((train_index_negative1_0_2st,train_index_negative1_1_3st)) 
train_index_negative1_24=np.concatenate((train_index_negative1_0_2st,train_index_negative1_1_4st)) 

##为每个二分类问题建立8个分类器，一共24个SVM分类器

In [82]:
svc_0=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_1=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_negative1=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 

svc_0.fit(train_data, train_label_0)
svc_1.fit(train_data, train_label_1)
svc_negative1.fit(train_data, train_label_negative1)

CalibratedClassifierCV(base_estimator=LinearSVC(C=2e-07, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv=3, method='sigmoid')

In [83]:
test_pred=[]
train_pred=[]
test_pred_0 = svc_0.predict_proba(test_data)
test_pred_1 = svc_1.predict_proba(test_data)
test_pred_negative1 = svc_negative1.predict_proba(test_data)
for i in range(13588):
    maxone=max(test_pred_0[i][0],test_pred_1[i][0],test_pred_negative1[i][0])
    if maxone == test_pred_0[i][0]:
        test_pred.append(0)
    elif maxone == test_pred_1[i][0]:
        test_pred.append(1)
    elif maxone == test_pred_negative1[i][0]:
        test_pred.append(-1)

In [84]:
train_pred_0 = svc_0.predict_proba(train_data)
train_pred_1 = svc_1.predict_proba(train_data)
train_pred_negative1 = svc_negative1.predict_proba(train_data)
for i in range(37367):
    maxone=max(train_pred_0[i][0],train_pred_1[i][0],train_pred_negative1[i][0])
    if maxone == train_pred_0[i][0]:
        train_pred.append(0)
    elif maxone == train_pred_1[i][0]:
        train_pred.append(1)
    elif maxone == train_pred_negative1[i][0]:
        train_pred.append(-1)

In [85]:
print(accuracy_score(test_label, test_pred))

0.581836914925


In [86]:
print(accuracy_score(train_label, train_pred))

0.603045467926


In [87]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(train_label, train_pred, target_names=target_names))

             precision    recall  f1-score   support

    class 0       0.63      0.47      0.54     12320
    class 1       0.54      0.59      0.56     12144
    class 2       0.64      0.74      0.69     12903

avg / total       0.61      0.60      0.60     37367



In [88]:
print(classification_report(test_label, test_pred, target_names=target_names))

             precision    recall  f1-score   support

    class 0       0.68      0.14      0.23      4480
    class 1       0.54      0.74      0.62      4416
    class 2       0.61      0.86      0.71      4692

avg / total       0.61      0.58      0.52     13588

